In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
from pathlib import Path

In [ ]:
def infection(S, I, R, N, iterations, I_comp, S_comp, R_comp, beta, gamma):
    for i in range(iterations):
        S, I, R = S - beta * ((S * I) / N), I + beta * ((S * I) / N) - gamma * I, R + gamma * I 
        S_comp.append(S)
        I_comp.append(I)
        R_comp.append(R)

In [ ]:
def SIR_mse_calc(N,contamine,susceptible,recovered):
    iterations = len(contamine)
    mse = 1000000000000
    betarange = np.linspace(0,100,101)
    gammarange = np.linspace(0,100,101)
    for pas_beta in betarange:
        beta = pas_beta/100
        for pas_gamma in gammarange:
            gamma = pas_gamma/100
            S = N - 1
            I = 1
            R = 0
            S_comp = [] # infected compartment
            I_comp = [] # susceptible compartment
            R_comp = [] # recovered compartment
            infection(S, I, R, N, iterations, I_comp, S_comp, R_comp, beta, gamma)
            newmsi = np.square(np.subtract(contamine['Nombre de contaminés'].values.tolist(),I_comp)).mean() + np.square(np.subtract(susceptible.values.tolist(),S_comp)).mean() + np.square(np.subtract(recovered['Recovered'].values.tolist(),R_comp)).mean()
            if(newmsi < mse):
                mse = newmsi
                betavalue = beta
                gammavalue = gamma
    return [mse,betavalue,gammavalue]

## Densité population 1/2

### 5000 individus

In [ ]:
path1 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/5000mouvements/densite1sur2/population5000"
path2 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/perfect_mix/densite1sur2/population5000"

In [ ]:
temps = pd.read_csv(path1 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

In [ ]:
contamine1 = pd.read_csv(path1 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered1 = pd.read_csv(path1 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)
contamine2 = pd.read_csv(path2 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered2 = pd.read_csv(path2 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)

N = 5000 # S + I = N
S = N - 1
I = 1
R = 0
beta = 1
gamma = 0.1
iterations = 100

S_comp = [] # infected compartment
I_comp = [] # susceptible compartment
R_comp = [] # recovered compartment

infection(S, I, R, N, iterations, I_comp, S_comp, R_comp, beta, gamma)
figure = plt.figure(figsize=(15, 15))

# Infected
plt.subplot(3,1,1)
I_SIR, = plt.plot(I_comp, label='I_SIR(t)')
I_5000, = plt.plot(contamine1, label='I_5000(t)')
I_MIX, = plt.plot(contamine2, label='I_MIX(t)')

plt.title("SIR_référence, densité 1/2, 5000 individus, Compartiment Infected")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[I_SIR, I_5000, I_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Recovered
plt.subplot(3,1,2)
R_SIR, = plt.plot(R_comp, label='R_SIR(t)')
R_5000, = plt.plot(recovered1, label='R_5000(t)')
R_MIX, = plt.plot(recovered2, label='R_MIX(t)')

plt.title("SIR_référence, densité 1/2, 5000 individus, Compartiment Recovered")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[R_SIR, R_5000, R_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Susceptible
plt.subplot(3,1,3)
S_SIR, = plt.plot(S_comp, label='S_SIR(t)')
S_5000, = plt.plot((N-contamine1)['Nombre de contaminés'].subtract(recovered1['Recovered']), label='S_5000(t)')
S_MIX, = plt.plot((N-contamine2)['Nombre de contaminés'].subtract(recovered2['Recovered']), label='S_MIX(t)')

plt.title("SIR_référence, densité 1/2, 5000 individus, Compartiment Susceptible")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[S_SIR, S_5000, S_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

plt.show()

### MSE

In [ ]:
susceptible1 = (N-contamine1)['Nombre de contaminés'].subtract(recovered1['Recovered'])
mse1 = SIR_mse_calc(N,contamine1,susceptible1,recovered1)
print("MSE minimal (SIR_5000,SIR_SIR) : ", mse1[0], " pour un paramètre beta = ", mse1[1], " et pour un paramètre gamma = ", mse1[2])

susceptible2 = (N-contamine2)['Nombre de contaminés'].subtract(recovered2['Recovered'])
mse2 = SIR_mse_calc(N,contamine2,susceptible2,recovered2)
print("MSE minimal (SIR_MIX,SIR_SIR) : ", mse2[0], " pour un paramètre beta = ", mse2[1], " et pour un paramètre gamma = ", mse2[2])

# Adapt length
if(len(contamine1) < len(contamine2)):
    contamine2 = contamine2[:len(contamine1)]
    recovered2 = recovered2[:len(recovered1)]
    susceptible2 = susceptible2[:len(susceptible1)]
else:
    contamine1 = contamine1[:len(contamine2)]
    recovered1 = recovered1[:len(recovered2)]
    susceptible1 = susceptible1[:len(susceptible2)]

print("MSE (I_5000,I_MIX) : ", np.square(np.subtract(contamine1['Nombre de contaminés'].values.tolist(),contamine2['Nombre de contaminés'].values.tolist())).mean() + np.square(np.subtract(recovered1['Recovered'].values.tolist(),recovered2['Recovered'].values.tolist())).mean() + np.square(np.subtract(susceptible1.values.tolist(),susceptible2.values.tolist())).mean())

### 20000 individus

In [ ]:
path1 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/5000mouvements/densite1sur2/population20000"
path2 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/perfect_mix/densite1sur2/population20000"

In [ ]:
temps = pd.read_csv(path1 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

In [ ]:
contamine1 = pd.read_csv(path1 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered1 = pd.read_csv(path1 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)
contamine2 = pd.read_csv(path2 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered2 = pd.read_csv(path2 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)

N = 20000 # S + I = N
S = N - 1
I = 1
R = 0
beta = 0.5
gamma = 0.1
iterations = 125

S_comp = [] # infected compartment
I_comp = [] # susceptible compartment
R_comp = [] # recovered compartment

infection(S, I, R, N, iterations, I_comp, S_comp, R_comp, beta, gamma)
figure = plt.figure(figsize=(15, 15))

# Infected
plt.subplot(3,1,1)
SIR_I, = plt.plot(I_comp, label='I_SIR(t)')
I_20000, = plt.plot(contamine1, label='I_20000(t)')
I_MIX, = plt.plot(contamine2, label='I_MIX(t)')

plt.title("SIR_référence, densité 1/2, 20000 individus, Compartiment Infected")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[I_SIR, I_20000, I_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Recovered
plt.subplot(3,1,2)
R_SIR, = plt.plot(R_comp, label='R_SIR(t)')
R_20000, = plt.plot(recovered1, label='R_20000(t)')
R_MIX, = plt.plot(recovered2, label='R_MIX(t)')

plt.title("SIR_référence, densité 1/2, 50000 individus, Compartiment Recovered")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[R_SIR, R_20000, I_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Susceptible
plt.subplot(3,1,3)
S_SIR, = plt.plot(S_comp, label='S_SIR(t)')
S_20000, = plt.plot((N-contamine1)['Nombre de contaminés'].subtract(recovered1['Recovered']), label='I_20000(t)')
S_MIX, = plt.plot((N-contamine2)['Nombre de contaminés'].subtract(recovered2['Recovered']), label='I_MIX(t)')

plt.title("SIR_référence, densité 1/2, 5000 individus, Compartiment Susceptible")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[S_SIR, S_20000, I_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

plt.show()

# MSE I
#print("MSE I : ", np.square(np.subtract(contamine['Nombre de contaminés'].values.tolist(),I_comp)).mean())

# MSE S
#print("MSE S : ", np.square(np.subtract(nombre_individus_data_S.values.tolist(),S_comp)).mean())

# MSE R
#print("MSE R : ", np.square(np.subtract(recovered['Recovered'].values.tolist(),R_comp)).mean())

### MSE

In [ ]:
susceptible1 = (N-contamine1)['Nombre de contaminés'].subtract(recovered1['Recovered'])
mse1 = SIR_mse_calc(N,contamine1,susceptible1,recovered1)
print("MSE minimal (SIR_5000,SIR_SIR) : ", mse1[0], " pour un paramètre beta = ", mse1[1], " et pour un paramètre gamma = ", mse1[2])

susceptible2 = (N-contamine2)['Nombre de contaminés'].subtract(recovered2['Recovered'])
mse2 = SIR_mse_calc(N,contamine2,susceptible2,recovered2)
print("MSE minimal (SIR_MIX,SIR_SIR) : ", mse2[0], " pour un paramètre beta = ", mse2[1], " et pour un paramètre gamma = ", mse2[2])

# Adapt length
if(len(contamine1) < len(contamine2)):
    contamine2 = contamine2[:len(contamine1)]
    recovered2 = recovered2[:len(recovered1)]
    susceptible2 = susceptible2[:len(susceptible1)]
else:
    contamine1 = contamine1[:len(contamine2)]
    recovered1 = recovered1[:len(recovered2)]
    susceptible1 = susceptible1[:len(susceptible2)]

print("MSE (I_5000,I_MIX) : ", np.square(np.subtract(contamine1['Nombre de contaminés'].values.tolist(),contamine2['Nombre de contaminés'].values.tolist())).mean() + np.square(np.subtract(recovered1['Recovered'].values.tolist(),recovered2['Recovered'].values.tolist())).mean() + np.square(np.subtract(susceptible1.values.tolist(),susceptible2.values.tolist())).mean())

### 50000 individus

In [ ]:
path1 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/5000mouvements/densite1sur2/population50000"
path2 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/perfect_mix/densite1sur2/population50000"

In [ ]:
temps = pd.read_csv(path1 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

In [ ]:
contamine1 = pd.read_csv(path1 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered1 = pd.read_csv(path1 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)
contamine2 = pd.read_csv(path2 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered2 = pd.read_csv(path2 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)

N = 50000 # S + I = N
S = N - 1
I = 1
R = 0
beta = 0.5
gamma = 0.1
iterations = 100

S_comp = [] # infected compartment
I_comp = [] # susceptible compartment
R_comp = [] # recovered compartment

infection(S, I, R, N, iterations, I_comp, S_comp, R_comp, beta, gamma)
figure = plt.figure(figsize=(15, 15))

# Infected
plt.subplot(3,1,1)
I_SIR, = plt.plot(I_comp, label='I_SIR(t)')
I_50000, = plt.plot(contamine1, label='I_50000(t)')
I_MIX, = plt.plot(contamine2, label='I_MIX(t)')

plt.title("SIR_référence, densité 1/2, 50000 individus, Compartiment Infected")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[I_SIR, I_50000, I_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Recovered
plt.subplot(3,1,2)
R_SIR, = plt.plot(R_comp, label='R_SIR(t)')
R_50000, = plt.plot(recovered1, label='R_50000(t)')
R_MIX, = plt.plot(recovered2, label='R_MIX(t)')

plt.title("SIR_référence, densité 1/2, 50000 individus, Compartiment Recovered")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[R_SIR, R_50000, R_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Susceptible
plt.subplot(3,1,3)
S_SIR, = plt.plot(S_comp, label='S_SIR(t)')
S_50000, = plt.plot((N-contamine1)['Nombre de contaminés'].subtract(recovered1['Recovered']), label='S_50000(t)')
S_MIX, = plt.plot((N-contamine2)['Nombre de contaminés'].subtract(recovered2['Recovered']), label='S_MIX(t)')

plt.title("SIR_référence, densité 1/2, 50000 individus, Compartiment Susceptible")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[S_SIR, S_50000, S_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

plt.show()

# MSE I
#print("MSE I : ", np.square(np.subtract(contamine['Nombre de contaminés'].values.tolist(),I_comp)).mean())

# MSE S
#print("MSE S : ", np.square(np.subtract(nombre_individus_data_S.values.tolist(),S_comp)).mean())

# MSE R
#print("MSE R : ", np.square(np.subtract(recovered['Recovered'].values.tolist(),R_comp)).mean())

### MSE

In [ ]:
susceptible1 = (N-contamine1)['Nombre de contaminés'].subtract(recovered1['Recovered'])
mse1 = SIR_mse_calc(N,contamine1,susceptible1,recovered1)
print("MSE minimal (SIR_5000,SIR_SIR) : ", mse1[0], " pour un paramètre beta = ", mse1[1], " et pour un paramètre gamma = ", mse1[2])

susceptible2 = (N-contamine2)['Nombre de contaminés'].subtract(recovered2['Recovered'])
mse2 = SIR_mse_calc(N,contamine2,susceptible2,recovered2)
print("MSE minimal (SIR_MIX,SIR_SIR) : ", mse2[0], " pour un paramètre beta = ", mse2[1], " et pour un paramètre gamma = ", mse2[2])

# Adapt length
if(len(contamine1) < len(contamine2)):
    contamine2 = contamine2[:len(contamine1)]
    recovered2 = recovered2[:len(recovered1)]
    susceptible2 = susceptible2[:len(susceptible1)]
else:
    contamine1 = contamine1[:len(contamine2)]
    recovered1 = recovered1[:len(recovered2)]
    susceptible1 = susceptible1[:len(susceptible2)]

print("MSE (I_5000,I_MIX) : ", np.square(np.subtract(contamine1['Nombre de contaminés'].values.tolist(),contamine2['Nombre de contaminés'].values.tolist())).mean() + np.square(np.subtract(recovered1['Recovered'].values.tolist(),recovered2['Recovered'].values.tolist())).mean() + np.square(np.subtract(susceptible1.values.tolist(),susceptible2.values.tolist())).mean())

### 100000 individus

In [ ]:
path1 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/5000mouvements/densite1sur2/population100000"
path2 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/perfect_mix/densite1sur2/population100000"

In [ ]:
temps = pd.read_csv(path1 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

In [ ]:
contamine1 = pd.read_csv(path1 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered1 = pd.read_csv(path1 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)
contamine2 = pd.read_csv(path2 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered2 = pd.read_csv(path2 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)

N = 100000 # S + I = N
S = N - 1
I = 1
R = 0
beta = 0.5
gamma = 0.1
iterations = 100

S_comp = [] # infected compartment
I_comp = [] # susceptible compartment
R_comp = [] # recovered compartment

infection(S, I, R, N, iterations, I_comp, S_comp, R_comp, beta, gamma)
figure = plt.figure(figsize=(15, 15))

# Infected
plt.subplot(3,1,1)
I_SIR, = plt.plot(I_comp, label='I_SIR(t)')
I_100000, = plt.plot(contamine1, label='I_100000(t)')
I_MIX, = plt.plot(contamine2, label='I_MIX(t)')

plt.title("SIR_référence, densité 1/2, 5000 individus, Compartiment Infected")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[I_SIR, I_100000, I_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Recovered
plt.subplot(3,1,2)
R_SIR, = plt.plot(R_comp, label='R_SIR(t)')
R_100000, = plt.plot(recovered1, label='R_100000(t)')
R_MIX, = plt.plot(recovered2, label='R_MIX(t)')

plt.title("SIR_référence, densité 1/2, 100000 individus, Compartiment Recovered")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[R_SIR, R_100000, R_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Susceptible
plt.subplot(3,1,3)
S_SIR, = plt.plot(S_comp, label='S_SIR(t)')
S_100000, = plt.plot((N-contamine1)['Nombre de contaminés'].subtract(recovered1['Recovered']), label='S_100000(t)')
S_MIX, = plt.plot((N-contamine2)['Nombre de contaminés'].subtract(recovered2['Recovered']), label='S_MIX(t)')

plt.title("SIR_référence, densité 1/2, 100000 individus, Compartiment Susceptible")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[S_SIR, S_100000, S_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

plt.show()

# MSE I
#print("MSE I : ", np.square(np.subtract(contamine['Nombre de contaminés'].values.tolist(),I_comp)).mean())

# MSE S
#print("MSE S : ", np.square(np.subtract(nombre_individus_data_S.values.tolist(),S_comp)).mean())

# MSE R
#print("MSE R : ", np.square(np.subtract(recovered['Recovered'].values.tolist(),R_comp)).mean())

### MSE

In [ ]:
susceptible1 = (N-contamine1)['Nombre de contaminés'].subtract(recovered1['Recovered'])
mse1 = SIR_mse_calc(N,contamine1,susceptible1,recovered1)
print("MSE minimal (SIR_5000,SIR_SIR) : ", mse1[0], " pour un paramètre beta = ", mse1[1], " et pour un paramètre gamma = ", mse1[2])

susceptible2 = (N-contamine2)['Nombre de contaminés'].subtract(recovered2['Recovered'])
mse2 = SIR_mse_calc(N,contamine2,susceptible2,recovered2)
print("MSE minimal (SIR_MIX,SIR_SIR) : ", mse2[0], " pour un paramètre beta = ", mse2[1], " et pour un paramètre gamma = ", mse2[2])

# Adapt length
if(len(contamine1) < len(contamine2)):
    contamine2 = contamine2[:len(contamine1)]
    recovered2 = recovered2[:len(recovered1)]
    susceptible2 = susceptible2[:len(susceptible1)]
else:
    contamine1 = contamine1[:len(contamine2)]
    recovered1 = recovered1[:len(recovered2)]
    susceptible1 = susceptible1[:len(susceptible2)]

print("MSE (I_5000,I_MIX) : ", np.square(np.subtract(contamine1['Nombre de contaminés'].values.tolist(),contamine2['Nombre de contaminés'].values.tolist())).mean() + np.square(np.subtract(recovered1['Recovered'].values.tolist(),recovered2['Recovered'].values.tolist())).mean() + np.square(np.subtract(susceptible1.values.tolist(),susceptible2.values.tolist())).mean())

## Densité population 1/4

### 5000 individus

In [ ]:
path1 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/5000mouvements/densite1sur4/population5000"
path2 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/perfect_mix/densite1sur4/population5000"

In [ ]:
temps = pd.read_csv(path1 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

In [ ]:
contamine1 = pd.read_csv(path1 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered1 = pd.read_csv(path1 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)
contamine2 = pd.read_csv(path2 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered2 = pd.read_csv(path2 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)

N = 5000 # S + I = N
S = N - 1
I = 1
R = 0
beta = 0.5
gamma = 0.1
iterations = 100

S_comp = [] # infected compartment
I_comp = [] # susceptible compartment
R_comp = [] # recovered compartment

infection(S, I, R, N, iterations, I_comp, S_comp, R_comp, beta, gamma)
figure = plt.figure(figsize=(15, 15))

# Infected
plt.subplot(3,1,1)
I_SIR, = plt.plot(I_comp, label='I_SIR(t)')
I_5000, = plt.plot(contamine1, label='I_5000(t)')
I_MIX, = plt.plot(contamine2, label='I_MIX(t)')

plt.title("SIR_référence, densité 1/4, 5000 individus, Compartiment Infected")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[I_SIR, I_5000, I_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Recovered
plt.subplot(3,1,2)
R_SIR, = plt.plot(R_comp, label='R_SIR(t)')
R_5000, = plt.plot(recovered1, label='R_5000(t)')
R_MIX, = plt.plot(recovered2, label='R_MIX(t)')

plt.title("SIR_référence, densité 1/4, 5000 individus, Compartiment Recovered")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[R_SIR, R_5000, R_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Susceptible
plt.subplot(3,1,3)
S_SIR, = plt.plot(S_comp, label='S_SIR(t)')
S_5000, = plt.plot((N-contamine1)['Nombre de contaminés'].subtract(recovered1['Recovered']), label='S_5000(t)')
S_MIX, = plt.plot((N-contamine2)['Nombre de contaminés'].subtract(recovered2['Recovered']), label='S_MIX(t)')

plt.title("SIR_référence, densité 1/4, 5000 individus, Compartiment Susceptible")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[S_SIR, S_5000, S_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

plt.show()

# MSE I
#print("MSE I : ", np.square(np.subtract(contamine['Nombre de contaminés'].values.tolist(),I_comp)).mean())

# MSE S
#print("MSE S : ", np.square(np.subtract(nombre_individus_data_S.values.tolist(),S_comp)).mean())

# MSE R
#print("MSE R : ", np.square(np.subtract(recovered['Recovered'].values.tolist(),R_comp)).mean())

### MSE

In [ ]:
susceptible1 = (N-contamine1)['Nombre de contaminés'].subtract(recovered1['Recovered'])
mse1 = SIR_mse_calc(N,contamine1,susceptible1,recovered1)
print("MSE minimal (SIR_5000,SIR_SIR) : ", mse1[0], " pour un paramètre beta = ", mse1[1], " et pour un paramètre gamma = ", mse1[2])

susceptible2 = (N-contamine2)['Nombre de contaminés'].subtract(recovered2['Recovered'])
mse2 = SIR_mse_calc(N,contamine2,susceptible2,recovered2)
print("MSE minimal (SIR_MIX,SIR_SIR) : ", mse2[0], " pour un paramètre beta = ", mse2[1], " et pour un paramètre gamma = ", mse2[2])

# Adapt length
if(len(contamine1) < len(contamine2)):
    contamine2 = contamine2[:len(contamine1)]
    recovered2 = recovered2[:len(recovered1)]
    susceptible2 = susceptible2[:len(susceptible1)]
else:
    contamine1 = contamine1[:len(contamine2)]
    recovered1 = recovered1[:len(recovered2)]
    susceptible1 = susceptible1[:len(susceptible2)]

print("MSE (I_5000,I_MIX) : ", np.square(np.subtract(contamine1['Nombre de contaminés'].values.tolist(),contamine2['Nombre de contaminés'].values.tolist())).mean() + np.square(np.subtract(recovered1['Recovered'].values.tolist(),recovered2['Recovered'].values.tolist())).mean() + np.square(np.subtract(susceptible1.values.tolist(),susceptible2.values.tolist())).mean())

### 20000 individus

In [ ]:
path1 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/5000mouvements/densite1sur4/population20000"
path2 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/perfect_mix/densite1sur4/population20000"

In [ ]:
temps = pd.read_csv(path1 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

In [ ]:
contamine1 = pd.read_csv(path1 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered1 = pd.read_csv(path1 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)
contamine2 = pd.read_csv(path2 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered2 = pd.read_csv(path2 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)

N = 20000 # S + I = N
S = N - 1
I = 1
R = 0
beta = 0.5
gamma = 0.1
iterations = 100

S_comp = [] # infected compartment
I_comp = [] # susceptible compartment
R_comp = [] # recovered compartment

infection(S, I, R, N, iterations, I_comp, S_comp, R_comp, beta, gamma)
figure = plt.figure(figsize=(15, 15))

# Infected
plt.subplot(3,1,1)
I_SIR, = plt.plot(I_comp, label='I_SIR(t)')
I_20000, = plt.plot(contamine1, label='I_20000(t)')
I_MIX, = plt.plot(contamine2, label='I_MIX(t)')

plt.title("SIR_référence, densité 1/4, 20000 individus, Compartiment Infected")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[I_SIR, I_20000, I_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Recovered
plt.subplot(3,1,2)
R_SIR, = plt.plot(R_comp, label='R_SIR(t)')
R_20000, = plt.plot(recovered1, label='R_20000(t)')
R_MIX, = plt.plot(recovered2, label='R_MIX(t)')

plt.title("SIR_référence, densité 1/4, 20000 individus, Compartiment Recovered")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[R_SIR, R_20000, R_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Susceptible
plt.subplot(3,1,3)
S_SIR, = plt.plot(S_comp, label='S_SIR(t)')
S_20000, = plt.plot((N-contamine1)['Nombre de contaminés'].subtract(recovered1['Recovered']), label='S_20000(t)')
S_MIX, = plt.plot((N-contamine2)['Nombre de contaminés'].subtract(recovered2['Recovered']), label='S_MIX(t)')

plt.title("SIR_référence, densité 1/4, 20000 individus, Compartiment Susceptible")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[S_SIR, S_20000, S_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

plt.show()

# MSE I
#print("MSE I : ", np.square(np.subtract(contamine['Nombre de contaminés'].values.tolist(),I_comp)).mean())

# MSE S
#print("MSE S : ", np.square(np.subtract(nombre_individus_data_S.values.tolist(),S_comp)).mean())

# MSE R
#print("MSE R : ", np.square(np.subtract(recovered['Recovered'].values.tolist(),R_comp)).mean())

### MSE

In [ ]:
susceptible1 = (N-contamine1)['Nombre de contaminés'].subtract(recovered1['Recovered'])
mse1 = SIR_mse_calc(N,contamine1,susceptible1,recovered1)
print("MSE minimal (SIR_5000,SIR_SIR) : ", mse1[0], " pour un paramètre beta = ", mse1[1], " et pour un paramètre gamma = ", mse1[2])

susceptible2 = (N-contamine2)['Nombre de contaminés'].subtract(recovered2['Recovered'])
mse2 = SIR_mse_calc(N,contamine2,susceptible2,recovered2)
print("MSE minimal (SIR_MIX,SIR_SIR) : ", mse2[0], " pour un paramètre beta = ", mse2[1], " et pour un paramètre gamma = ", mse2[2])

# Adapt length
if(len(contamine1) < len(contamine2)):
    contamine2 = contamine2[:len(contamine1)]
    recovered2 = recovered2[:len(recovered1)]
    susceptible2 = susceptible2[:len(susceptible1)]
else:
    contamine1 = contamine1[:len(contamine2)]
    recovered1 = recovered1[:len(recovered2)]
    susceptible1 = susceptible1[:len(susceptible2)]

print("MSE (I_5000,I_MIX) : ", np.square(np.subtract(contamine1['Nombre de contaminés'].values.tolist(),contamine2['Nombre de contaminés'].values.tolist())).mean() + np.square(np.subtract(recovered1['Recovered'].values.tolist(),recovered2['Recovered'].values.tolist())).mean() + np.square(np.subtract(susceptible1.values.tolist(),susceptible2.values.tolist())).mean())

### 50000 individus

In [ ]:
path1 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/5000mouvements/densite1sur4/population50000"
path2 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/perfect_mix/densite1sur4/population50000"

In [ ]:
temps = pd.read_csv(path1 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

In [ ]:
contamine1 = pd.read_csv(path1 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered1 = pd.read_csv(path1 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)
contamine2 = pd.read_csv(path2 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered2 = pd.read_csv(path2 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)

N = 50000 # S + I = N
S = N - 1
I = 1
R = 0
beta = 0.5
gamma = 0.1
iterations = 100

S_comp = [] # infected compartment
I_comp = [] # susceptible compartment
R_comp = [] # recovered compartment

infection(S, I, R, N, iterations, I_comp, S_comp, R_comp, beta, gamma)
figure = plt.figure(figsize=(15, 15))

# Infected
plt.subplot(3,1,1)
I_SIR, = plt.plot(I_comp, label='I_SIR(t)')
I_50000, = plt.plot(contamine1, label='I_50000(t)')
I_MIX, = plt.plot(contamine2, label='I_MIX(t)')

plt.title("SIR_référence, densité 1/4, 50000 individus, Compartiment Infected")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[I_SIR, I_50000, I_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Recovered
plt.subplot(3,1,2)
R_SIR, = plt.plot(R_comp, label='R_SIR(t)')
R_50000, = plt.plot(recovered1, label='R_50000(t)')
R_MIX, = plt.plot(recovered2, label='R_MIX(t)')

plt.title("SIR_référence, densité 1/4, 50000 individus, Compartiment Recovered")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[R_SIR, R_50000, R_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Susceptible
plt.subplot(3,1,3)
S_SIR, = plt.plot(S_comp, label='S_SIR(t)')
S_50000, = plt.plot((N-contamine1)['Nombre de contaminés'].subtract(recovered1['Recovered']), label='S_50000(t)')
S_MIX, = plt.plot((N-contamine2)['Nombre de contaminés'].subtract(recovered2['Recovered']), label='S_MIX(t)')

plt.title("SIR_référence, densité 1/4, 50000 individus, Compartiment Susceptible")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[S_SIR, S_50000, S_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

plt.show()

# MSE I
#print("MSE I : ", np.square(np.subtract(contamine['Nombre de contaminés'].values.tolist(),I_comp)).mean())

# MSE S
#print("MSE S : ", np.square(np.subtract(nombre_individus_data_S.values.tolist(),S_comp)).mean())

# MSE R
#print("MSE R : ", np.square(np.subtract(recovered['Recovered'].values.tolist(),R_comp)).mean())

### MSE

In [ ]:
susceptible1 = (N-contamine1)['Nombre de contaminés'].subtract(recovered1['Recovered'])
mse1 = SIR_mse_calc(N,contamine1,susceptible1,recovered1)
print("MSE minimal (SIR_5000,SIR_SIR) : ", mse1[0], " pour un paramètre beta = ", mse1[1], " et pour un paramètre gamma = ", mse1[2])

susceptible2 = (N-contamine2)['Nombre de contaminés'].subtract(recovered2['Recovered'])
mse2 = SIR_mse_calc(N,contamine2,susceptible2,recovered2)
print("MSE minimal (SIR_MIX,SIR_SIR) : ", mse2[0], " pour un paramètre beta = ", mse2[1], " et pour un paramètre gamma = ", mse2[2])

# Adapt length
if(len(contamine1) < len(contamine2)):
    contamine2 = contamine2[:len(contamine1)]
    recovered2 = recovered2[:len(recovered1)]
    susceptible2 = susceptible2[:len(susceptible1)]
else:
    contamine1 = contamine1[:len(contamine2)]
    recovered1 = recovered1[:len(recovered2)]
    susceptible1 = susceptible1[:len(susceptible2)]

print("MSE (I_5000,I_MIX) : ", np.square(np.subtract(contamine1['Nombre de contaminés'].values.tolist(),contamine2['Nombre de contaminés'].values.tolist())).mean() + np.square(np.subtract(recovered1['Recovered'].values.tolist(),recovered2['Recovered'].values.tolist())).mean() + np.square(np.subtract(susceptible1.values.tolist(),susceptible2.values.tolist())).mean())

### 100000 individus

In [ ]:
path1 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/5000mouvements/densite1sur4/population100000"
path2 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/perfect_mix/densite1sur4/population100000"

In [ ]:
temps = pd.read_csv(path1 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

In [ ]:
contamine1 = pd.read_csv(path1 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered1 = pd.read_csv(path1 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)
contamine2 = pd.read_csv(path2 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered2 = pd.read_csv(path2 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)

N = 100000 # S + I = N
S = N - 1
I = 1
R = 0
beta = 0.5
gamma = 0.1
iterations = 100

S_comp = [] # infected compartment
I_comp = [] # susceptible compartment
R_comp = [] # recovered compartment

infection(S, I, R, N, iterations, I_comp, S_comp, R_comp, beta, gamma)
figure = plt.figure(figsize=(15, 15))

# Infected
plt.subplot(3,1,1)
I_SIR, = plt.plot(I_comp, label='I_SIR(t)')
I_100000, = plt.plot(contamine1, label='I_100000(t)')
I_MIX, = plt.plot(contamine2, label='I_MIX(t)')

plt.title("SIR_référence, densité 1/4, 100000 individus, Compartiment Infected")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[I_SIR, I_100000, I_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Recovered
plt.subplot(3,1,2)
R_SIR, = plt.plot(R_comp, label='R_SIR(t)')
R_100000, = plt.plot(recovered1, label='R_100000(t)')
R_MIX, = plt.plot(recovered2, label='R_MIX(t)')

plt.title("SIR_référence, densité 1/4, 100000 individus, Compartiment Recovered")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[R_SIR, R_100000, R_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Susceptible
plt.subplot(3,1,3)
S_SIR, = plt.plot(S_comp, label='S_SIR(t)')
S_100000, = plt.plot((N-contamine1)['Nombre de contaminés'].subtract(recovered1['Recovered']), label='S_100000(t)')
S_MIX, = plt.plot((N-contamine2)['Nombre de contaminés'].subtract(recovered2['Recovered']), label='S_MIX(t)')

plt.title("SIR_référence, densité 1/4, 100000 individus, Compartiment Susceptible")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[S_SIR, S_100000, S_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

plt.show()

# MSE I
#print("MSE I : ", np.square(np.subtract(contamine['Nombre de contaminés'].values.tolist(),I_comp)).mean())

# MSE S
#print("MSE S : ", np.square(np.subtract(nombre_individus_data_S.values.tolist(),S_comp)).mean())

# MSE R
#print("MSE R : ", np.square(np.subtract(recovered['Recovered'].values.tolist(),R_comp)).mean())

### MSE

In [ ]:
susceptible1 = (N-contamine1)['Nombre de contaminés'].subtract(recovered1['Recovered'])
mse1 = SIR_mse_calc(N,contamine1,susceptible1,recovered1)
print("MSE minimal (SIR_5000,SIR_SIR) : ", mse1[0], " pour un paramètre beta = ", mse1[1], " et pour un paramètre gamma = ", mse1[2])

susceptible2 = (N-contamine2)['Nombre de contaminés'].subtract(recovered2['Recovered'])
mse2 = SIR_mse_calc(N,contamine2,susceptible2,recovered2)
print("MSE minimal (SIR_MIX,SIR_SIR) : ", mse2[0], " pour un paramètre beta = ", mse2[1], " et pour un paramètre gamma = ", mse2[2])

# Adapt length
if(len(contamine1) < len(contamine2)):
    contamine2 = contamine2[:len(contamine1)]
    recovered2 = recovered2[:len(recovered1)]
    susceptible2 = susceptible2[:len(susceptible1)]
else:
    contamine1 = contamine1[:len(contamine2)]
    recovered1 = recovered1[:len(recovered2)]
    susceptible1 = susceptible1[:len(susceptible2)]

print("MSE (I_5000,I_MIX) : ", np.square(np.subtract(contamine1['Nombre de contaminés'].values.tolist(),contamine2['Nombre de contaminés'].values.tolist())).mean() + np.square(np.subtract(recovered1['Recovered'].values.tolist(),recovered2['Recovered'].values.tolist())).mean() + np.square(np.subtract(susceptible1.values.tolist(),susceptible2.values.tolist())).mean())

## Densité population 1/8

### 5000 individus

In [ ]:
path1 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/5000mouvements/densite1sur8/population5000"
path2 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/perfect_mix/densite1sur8/population5000"

In [ ]:
temps = pd.read_csv(path1 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

In [ ]:
contamine1 = pd.read_csv(path1 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered1 = pd.read_csv(path1 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)
contamine2 = pd.read_csv(path2 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered2 = pd.read_csv(path2 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)

N = 5000 # S + I = N
S = N - 1
I = 1
R = 0
beta = 0.5
gamma = 0.1
iterations = 100

S_comp = [] # infected compartment
I_comp = [] # susceptible compartment
R_comp = [] # recovered compartment

infection(S, I, R, N, iterations, I_comp, S_comp, R_comp, beta, gamma)
figure = plt.figure(figsize=(15, 15))

# Infected
plt.subplot(3,1,1)
I_SIR, = plt.plot(I_comp, label='I_SIR(t)')
I_5000, = plt.plot(contamine1, label='I_5000(t)')
I_MIX, = plt.plot(contamine2, label='I_MIX(t)')

plt.title("SIR_référence, densité 1/8, 5000 individus, Compartiment Infected")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[I_SIR, I_5000, I_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Recovered
plt.subplot(3,1,2)
R_SIR, = plt.plot(R_comp, label='R_SIR(t)')
R_5000, = plt.plot(recovered1, label='R_5000(t)')
R_MIX, = plt.plot(recovered2, label='R_MIX(t)')

plt.title("SIR_référence, densité 1/8, 5000 individus, Compartiment Recovered")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[R_SIR, R_5000, R_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Susceptible
plt.subplot(3,1,3)
S_SIR, = plt.plot(S_comp, label='S_SIR(t)')
S_5000, = plt.plot((N-contamine1)['Nombre de contaminés'].subtract(recovered1['Recovered']), label='S_5000(t)')
S_MIX, = plt.plot((N-contamine2)['Nombre de contaminés'].subtract(recovered2['Recovered']), label='S_MIX(t)')

plt.title("SIR_référence, densité 1/8, 5000 individus, Compartiment Susceptible")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[S_SIR, S_5000, S_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

plt.show()

# MSE I
#print("MSE I : ", np.square(np.subtract(contamine['Nombre de contaminés'].values.tolist(),I_comp)).mean())

# MSE S
#print("MSE S : ", np.square(np.subtract(nombre_individus_data_S.values.tolist(),S_comp)).mean())

# MSE R
#print("MSE R : ", np.square(np.subtract(recovered['Recovered'].values.tolist(),R_comp)).mean())

### MSE

In [ ]:
susceptible1 = (N-contamine1)['Nombre de contaminés'].subtract(recovered1['Recovered'])
mse1 = SIR_mse_calc(N,contamine1,susceptible1,recovered1)
print("MSE minimal (SIR_5000,SIR_SIR) : ", mse1[0], " pour un paramètre beta = ", mse1[1], " et pour un paramètre gamma = ", mse1[2])

susceptible2 = (N-contamine2)['Nombre de contaminés'].subtract(recovered2['Recovered'])
mse2 = SIR_mse_calc(N,contamine2,susceptible2,recovered2)
print("MSE minimal (SIR_MIX,SIR_SIR) : ", mse2[0], " pour un paramètre beta = ", mse2[1], " et pour un paramètre gamma = ", mse2[2])

# Adapt length
if(len(contamine1) < len(contamine2)):
    contamine2 = contamine2[:len(contamine1)]
    recovered2 = recovered2[:len(recovered1)]
    susceptible2 = susceptible2[:len(susceptible1)]
else:
    contamine1 = contamine1[:len(contamine2)]
    recovered1 = recovered1[:len(recovered2)]
    susceptible1 = susceptible1[:len(susceptible2)]

print("MSE (I_5000,I_MIX) : ", np.square(np.subtract(contamine1['Nombre de contaminés'].values.tolist(),contamine2['Nombre de contaminés'].values.tolist())).mean() + np.square(np.subtract(recovered1['Recovered'].values.tolist(),recovered2['Recovered'].values.tolist())).mean() + np.square(np.subtract(susceptible1.values.tolist(),susceptible2.values.tolist())).mean())

### 20000 individus

In [ ]:
path1 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/5000mouvements/densite1sur8/population20000"
path2 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/perfect_mix/densite1sur8/population20000"

In [ ]:
temps = pd.read_csv(path1 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

In [ ]:
contamine1 = pd.read_csv(path1 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered1 = pd.read_csv(path1 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)
contamine2 = pd.read_csv(path2 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered2 = pd.read_csv(path2 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)

N = 20000 # S + I = N
S = N - 1
I = 1
R = 0
beta = 0.5
gamma = 0.1
iterations = 100

S_comp = [] # infected compartment
I_comp = [] # susceptible compartment
R_comp = [] # recovered compartment

infection(S, I, R, N, iterations, I_comp, S_comp, R_comp, beta, gamma)
figure = plt.figure(figsize=(15, 15))

# Infected
plt.subplot(3,1,1)
I_SIR, = plt.plot(I_comp, label='I_SIR(t)')
I_20000, = plt.plot(contamine1, label='I_20000(t)')
I_MIX, = plt.plot(contamine2, label='I_MIX(t)')

plt.title("SIR_référence, densité 1/8, 20000 individus, Compartiment Infected")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[I_SIR, I_20000, I_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Recovered
plt.subplot(3,1,2)
R_SIR, = plt.plot(R_comp, label='R_SIR(t)')
R_20000, = plt.plot(recovered1, label='R_20000(t)')
R_MIX, = plt.plot(recovered2, label='R_MIX(t)')

plt.title("SIR_référence, densité 1/8, 20000 individus, Compartiment Recovered")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[R_SIR, R_20000, R_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Susceptible
plt.subplot(3,1,3)
S_SIR, = plt.plot(S_comp, label='S_SIR(t)')
S_20000, = plt.plot((N-contamine1)['Nombre de contaminés'].subtract(recovered1['Recovered']), label='S_20000(t)')
S_MIX, = plt.plot((N-contamine2)['Nombre de contaminés'].subtract(recovered2['Recovered']), label='S_MIX(t)')

plt.title("SIR_référence, densité 1/8, 20000 individus, Compartiment Susceptible")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[S_SIR, S_20000, S_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

plt.show()

# MSE I
#print("MSE I : ", np.square(np.subtract(contamine['Nombre de contaminés'].values.tolist(),I_comp)).mean())

# MSE S
#print("MSE S : ", np.square(np.subtract(nombre_individus_data_S.values.tolist(),S_comp)).mean())

# MSE R
#print("MSE R : ", np.square(np.subtract(recovered['Recovered'].values.tolist(),R_comp)).mean())

### MSE

In [ ]:
susceptible1 = (N-contamine1)['Nombre de contaminés'].subtract(recovered1['Recovered'])
mse1 = SIR_mse_calc(N,contamine1,susceptible1,recovered1)
print("MSE minimal (SIR_5000,SIR_SIR) : ", mse1[0], " pour un paramètre beta = ", mse1[1], " et pour un paramètre gamma = ", mse1[2])

susceptible2 = (N-contamine2)['Nombre de contaminés'].subtract(recovered2['Recovered'])
mse2 = SIR_mse_calc(N,contamine2,susceptible2,recovered2)
print("MSE minimal (SIR_MIX,SIR_SIR) : ", mse2[0], " pour un paramètre beta = ", mse2[1], " et pour un paramètre gamma = ", mse2[2])

# Adapt length
if(len(contamine1) < len(contamine2)):
    contamine2 = contamine2[:len(contamine1)]
    recovered2 = recovered2[:len(recovered1)]
    susceptible2 = susceptible2[:len(susceptible1)]
else:
    contamine1 = contamine1[:len(contamine2)]
    recovered1 = recovered1[:len(recovered2)]
    susceptible1 = susceptible1[:len(susceptible2)]

print("MSE (I_5000,I_MIX) : ", np.square(np.subtract(contamine1['Nombre de contaminés'].values.tolist(),contamine2['Nombre de contaminés'].values.tolist())).mean() + np.square(np.subtract(recovered1['Recovered'].values.tolist(),recovered2['Recovered'].values.tolist())).mean() + np.square(np.subtract(susceptible1.values.tolist(),susceptible2.values.tolist())).mean())

### 50000 individus

In [ ]:
path1 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/5000mouvements/densite1sur8/population50000"
path2 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/perfect_mix/densite1sur8/population50000"

In [ ]:
temps = pd.read_csv(path1 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

In [ ]:
contamine1 = pd.read_csv(path1 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered1 = pd.read_csv(path1 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)
contamine2 = pd.read_csv(path2 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered2 = pd.read_csv(path2 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)

N = 50000 # S + I = N
S = N - 1
I = 1
R = 0
beta = 0.5
gamma = 0.1
iterations = 100

S_comp = [] # infected compartment
I_comp = [] # susceptible compartment
R_comp = [] # recovered compartment

infection(S, I, R, N, iterations, I_comp, S_comp, R_comp, beta, gamma)
figure = plt.figure(figsize=(15, 15))

# Infected
plt.subplot(3,1,1)
I_SIR, = plt.plot(I_comp, label='I_SIR(t)')
I_50000, = plt.plot(contamine1, label='I_50000(t)')
I_MIX, = plt.plot(contamine2, label='I_MIX(t)')

plt.title("SIR_référence, densité 1/8, 50000 individus, Compartiment Infected")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[I_SIR, I_50000, I_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Recovered
plt.subplot(3,1,2)
R_SIR, = plt.plot(R_comp, label='R_SIR(t)')
R_50000, = plt.plot(recovered1, label='R_50000(t)')
R_MIX, = plt.plot(recovered2, label='R_MIX(t)')

plt.title("SIR_référence, densité 1/8, 50000 individus, Compartiment Recovered")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[R_SIR, R_50000, R_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Susceptible
plt.subplot(3,1,3)
S_SIR, = plt.plot(S_comp, label='S_SIR(t)')
S_50000, = plt.plot((N-contamine1)['Nombre de contaminés'].subtract(recovered1['Recovered']), label='S_50000(t)')
S_MIX, = plt.plot((N-contamine2)['Nombre de contaminés'].subtract(recovered2['Recovered']), label='S_MIX(t)')

plt.title("SIR_référence, densité 1/8, 50000 individus, Compartiment Susceptible")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[S_SIR, S_50000, S_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

plt.show()

# MSE I
#print("MSE I : ", np.square(np.subtract(contamine['Nombre de contaminés'].values.tolist(),I_comp)).mean())

# MSE S
#print("MSE S : ", np.square(np.subtract(nombre_individus_data_S.values.tolist(),S_comp)).mean())

# MSE R
#print("MSE R : ", np.square(np.subtract(recovered['Recovered'].values.tolist(),R_comp)).mean())

### MSE

In [ ]:
susceptible1 = (N-contamine1)['Nombre de contaminés'].subtract(recovered1['Recovered'])
mse1 = SIR_mse_calc(N,contamine1,susceptible1,recovered1)
print("MSE minimal (SIR_5000,SIR_SIR) : ", mse1[0], " pour un paramètre beta = ", mse1[1], " et pour un paramètre gamma = ", mse1[2])

susceptible2 = (N-contamine2)['Nombre de contaminés'].subtract(recovered2['Recovered'])
mse2 = SIR_mse_calc(N,contamine2,susceptible2,recovered2)
print("MSE minimal (SIR_MIX,SIR_SIR) : ", mse2[0], " pour un paramètre beta = ", mse2[1], " et pour un paramètre gamma = ", mse2[2])

# Adapt length
if(len(contamine1) < len(contamine2)):
    contamine2 = contamine2[:len(contamine1)]
    recovered2 = recovered2[:len(recovered1)]
    susceptible2 = susceptible2[:len(susceptible1)]
else:
    contamine1 = contamine1[:len(contamine2)]
    recovered1 = recovered1[:len(recovered2)]
    susceptible1 = susceptible1[:len(susceptible2)]

print("MSE (I_5000,I_MIX) : ", np.square(np.subtract(contamine1['Nombre de contaminés'].values.tolist(),contamine2['Nombre de contaminés'].values.tolist())).mean() + np.square(np.subtract(recovered1['Recovered'].values.tolist(),recovered2['Recovered'].values.tolist())).mean() + np.square(np.subtract(susceptible1.values.tolist(),susceptible2.values.tolist())).mean())

### 100000 individus

In [ ]:
path1 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/5000mouvements/densite1sur8/population100000"
path2 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/perfect_mix/densite1sur8/population100000"

In [ ]:
temps = pd.read_csv(path1 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

In [ ]:
contamine1 = pd.read_csv(path1 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered1 = pd.read_csv(path1 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)
contamine2 = pd.read_csv(path2 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered2 = pd.read_csv(path2 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)

N = 100000 # S + I = N
S = N - 1
I = 1
R = 0
beta = 0.5
gamma = 0.1
iterations = 100

S_comp = [] # infected compartment
I_comp = [] # susceptible compartment
R_comp = [] # recovered compartment

infection(S, I, R, N, iterations, I_comp, S_comp, R_comp, beta, gamma)
figure = plt.figure(figsize=(15, 15))

# Infected
plt.subplot(3,1,1)
I_SIR, = plt.plot(I_comp, label='I_SIR(t)')
I_100000, = plt.plot(contamine1, label='I_100000(t)')
I_MIX, = plt.plot(contamine2, label='I_MIX(t)')

plt.title("SIR_référence, densité 1/8, 100000 individus, Compartiment Infected")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[I_SIR, I_100000, I_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Recovered
plt.subplot(3,1,2)
R_SIR, = plt.plot(R_comp, label='R_SIR(t)')
R_100000, = plt.plot(recovered1, label='R_100000(t)')
R_MIX, = plt.plot(recovered2, label='R_MIX(t)')

plt.title("SIR_référence, densité 1/8, 100000 individus, Compartiment Recovered")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[R_SIR, R_100000, R_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Susceptible
plt.subplot(3,1,3)
S_SIR, = plt.plot(S_comp, label='S_SIR(t)')
S_100000, = plt.plot((N-contamine1)['Nombre de contaminés'].subtract(recovered1['Recovered']), label='S_100000(t)')
S_MIX, = plt.plot((N-contamine2)['Nombre de contaminés'].subtract(recovered2['Recovered']), label='S_MIX(t)')

plt.title("SIR_référence, densité 1/8, 100000 individus, Compartiment Susceptible")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[S_SIR, S_100000, S_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

plt.show()

# MSE I
#print("MSE I : ", np.square(np.subtract(contamine['Nombre de contaminés'].values.tolist(),I_comp)).mean())

# MSE S
#print("MSE S : ", np.square(np.subtract(nombre_individus_data_S.values.tolist(),S_comp)).mean())

# MSE R
#print("MSE R : ", np.square(np.subtract(recovered['Recovered'].values.tolist(),R_comp)).mean())

### MSE

In [ ]:
susceptible1 = (N-contamine1)['Nombre de contaminés'].subtract(recovered1['Recovered'])
mse1 = SIR_mse_calc(N,contamine1,susceptible1,recovered1)
print("MSE minimal (SIR_5000,SIR_SIR) : ", mse1[0], " pour un paramètre beta = ", mse1[1], " et pour un paramètre gamma = ", mse1[2])

susceptible2 = (N-contamine2)['Nombre de contaminés'].subtract(recovered2['Recovered'])
mse2 = SIR_mse_calc(N,contamine2,susceptible2,recovered2)
print("MSE minimal (SIR_MIX,SIR_SIR) : ", mse2[0], " pour un paramètre beta = ", mse2[1], " et pour un paramètre gamma = ", mse2[2])

# Adapt length
if(len(contamine1) < len(contamine2)):
    contamine2 = contamine2[:len(contamine1)]
    recovered2 = recovered2[:len(recovered1)]
    susceptible2 = susceptible2[:len(susceptible1)]
else:
    contamine1 = contamine1[:len(contamine2)]
    recovered1 = recovered1[:len(recovered2)]
    susceptible1 = susceptible1[:len(susceptible2)]

print("MSE (I_5000,I_MIX) : ", np.square(np.subtract(contamine1['Nombre de contaminés'].values.tolist(),contamine2['Nombre de contaminés'].values.tolist())).mean() + np.square(np.subtract(recovered1['Recovered'].values.tolist(),recovered2['Recovered'].values.tolist())).mean() + np.square(np.subtract(susceptible1.values.tolist(),susceptible2.values.tolist())).mean())

## Densité population 1/16

### 5000 individus

In [ ]:
path1 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/5000mouvements/densite1sur16/population5000"
path2 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/perfect_mix/densite1sur16/population5000"

In [ ]:
temps = pd.read_csv(path1 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

In [ ]:
contamine1 = pd.read_csv(path1 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered1 = pd.read_csv(path1 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)
contamine2 = pd.read_csv(path2 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered2 = pd.read_csv(path2 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)

N = 5000 # S + I = N
S = N - 1
I = 1
R = 0
beta = 0.5
gamma = 0.1
iterations = 100

S_comp = [] # infected compartment
I_comp = [] # susceptible compartment
R_comp = [] # recovered compartment

infection(S, I, R, N, iterations, I_comp, S_comp, R_comp, beta, gamma)
figure = plt.figure(figsize=(15, 15))

# Infected
plt.subplot(3,1,1)
I_SIR, = plt.plot(I_comp, label='I_SIR(t)')
I_5000, = plt.plot(contamine1, label='I_5000(t)')
I_MIX, = plt.plot(contamine2, label='I_MIX(t)')

plt.title("SIR_référence, densité 1/16, 5000 individus, Compartiment Infected")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[I_SIR, I_5000, I_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Recovered
plt.subplot(3,1,2)
R_SIR, = plt.plot(R_comp, label='R_SIR(t)')
R_5000, = plt.plot(recovered1, label='R_5000(t)')
R_MIX, = plt.plot(recovered2, label='R_MIX(t)')

plt.title("SIR_référence, densité 1/16, 5000 individus, Compartiment Recovered")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[R_SIR, R_5000, R_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Susceptible
plt.subplot(3,1,3)
S_SIR, = plt.plot(S_comp, label='S_SIR(t)')
S_5000, = plt.plot((N-contamine1)['Nombre de contaminés'].subtract(recovered1['Recovered']), label='S_5000(t)')
S_MIX, = plt.plot((N-contamine2)['Nombre de contaminés'].subtract(recovered2['Recovered']), label='S_MIX(t)')

plt.title("SIR_référence, densité 1/16, 5000 individus, Compartiment Susceptible")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[S_SIR, S_5000, S_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

plt.show()

### MSE

In [ ]:
susceptible1 = (N-contamine1)['Nombre de contaminés'].subtract(recovered1['Recovered'])
mse1 = SIR_mse_calc(N,contamine1,susceptible1,recovered1)
print("MSE minimal (SIR_5000,SIR_SIR) : ", mse1[0], " pour un paramètre beta = ", mse1[1], " et pour un paramètre gamma = ", mse1[2])

susceptible2 = (N-contamine2)['Nombre de contaminés'].subtract(recovered2['Recovered'])
mse2 = SIR_mse_calc(N,contamine2,susceptible2,recovered2)
print("MSE minimal (SIR_MIX,SIR_SIR) : ", mse2[0], " pour un paramètre beta = ", mse2[1], " et pour un paramètre gamma = ", mse2[2])

# Adapt length
if(len(contamine1) < len(contamine2)):
    contamine2 = contamine2[:len(contamine1)]
    recovered2 = recovered2[:len(recovered1)]
    susceptible2 = susceptible2[:len(susceptible1)]
else:
    contamine1 = contamine1[:len(contamine2)]
    recovered1 = recovered1[:len(recovered2)]
    susceptible1 = susceptible1[:len(susceptible2)]

print("MSE (I_5000,I_MIX) : ", np.square(np.subtract(contamine1['Nombre de contaminés'].values.tolist(),contamine2['Nombre de contaminés'].values.tolist())).mean() + np.square(np.subtract(recovered1['Recovered'].values.tolist(),recovered2['Recovered'].values.tolist())).mean() + np.square(np.subtract(susceptible1.values.tolist(),susceptible2.values.tolist())).mean())

### 20000 individus

In [ ]:
path1 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/5000mouvements/densite1sur16/population20000"
path2 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/perfect_mix/densite1sur16/population20000"

In [ ]:
temps = pd.read_csv(path1 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

In [ ]:
contamine1 = pd.read_csv(path1 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered1 = pd.read_csv(path1 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)
contamine2 = pd.read_csv(path2 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered2 = pd.read_csv(path2 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)

N = 20000 # S + I = N
S = N - 1
I = 1
R = 0
beta = 0.5
gamma = 0.1
iterations = 100

S_comp = [] # infected compartment
I_comp = [] # susceptible compartment
R_comp = [] # recovered compartment

infection(S, I, R, N, iterations, I_comp, S_comp, R_comp, beta, gamma)
figure = plt.figure(figsize=(15, 15))

# Infected
plt.subplot(3,1,1)
I_SIR, = plt.plot(I_comp, label='I_SIR(t)')
I_20000, = plt.plot(contamine1, label='I_20000(t)')
I_MIX, = plt.plot(contamine2, label='I_MIX(t)')

plt.title("SIR_référence, densité 1/16, 20000 individus, Compartiment Infected")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[I_SIR, I_20000, I_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Recovered
plt.subplot(3,1,2)
R_SIR, = plt.plot(R_comp, label='R_SIR(t)')
R_20000, = plt.plot(recovered1, label='R_20000(t)')
R_MIX, = plt.plot(recovered2, label='R_MIX(t)')

plt.title("SIR_référence, densité 1/16, 20000 individus, Compartiment Recovered")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[R_SIR, R_20000, R_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Susceptible
plt.subplot(3,1,3)
S_SIR, = plt.plot(S_comp, label='S_SIR(t)')
S_20000, = plt.plot((N-contamine1)['Nombre de contaminés'].subtract(recovered1['Recovered']), label='S_20000(t)')
S_MIX, = plt.plot((N-contamine2)['Nombre de contaminés'].subtract(recovered2['Recovered']), label='S_MIX(t)')

plt.title("SIR_référence, densité 1/16, 20000 individus, Compartiment Susceptible")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[S_SIR, S_20000, S_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

plt.show()

### MSE

In [ ]:
susceptible1 = (N-contamine1)['Nombre de contaminés'].subtract(recovered1['Recovered'])
mse1 = SIR_mse_calc(N,contamine1,susceptible1,recovered1)
print("MSE minimal (SIR_5000,SIR_SIR) : ", mse1[0], " pour un paramètre beta = ", mse1[1], " et pour un paramètre gamma = ", mse1[2])

susceptible2 = (N-contamine2)['Nombre de contaminés'].subtract(recovered2['Recovered'])
mse2 = SIR_mse_calc(N,contamine2,susceptible2,recovered2)
print("MSE minimal (SIR_MIX,SIR_SIR) : ", mse2[0], " pour un paramètre beta = ", mse2[1], " et pour un paramètre gamma = ", mse2[2])

# Adapt length
if(len(contamine1) < len(contamine2)):
    contamine2 = contamine2[:len(contamine1)]
    recovered2 = recovered2[:len(recovered1)]
    susceptible2 = susceptible2[:len(susceptible1)]
else:
    contamine1 = contamine1[:len(contamine2)]
    recovered1 = recovered1[:len(recovered2)]
    susceptible1 = susceptible1[:len(susceptible2)]

print("MSE (I_5000,I_MIX) : ", np.square(np.subtract(contamine1['Nombre de contaminés'].values.tolist(),contamine2['Nombre de contaminés'].values.tolist())).mean() + np.square(np.subtract(recovered1['Recovered'].values.tolist(),recovered2['Recovered'].values.tolist())).mean() + np.square(np.subtract(susceptible1.values.tolist(),susceptible2.values.tolist())).mean())

### 50000 individus

In [ ]:
path1 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/5000mouvements/densite1sur16/population50000"
path2 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/perfect_mix/densite1sur16/population50000"

In [ ]:
temps = pd.read_csv(path1 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

In [ ]:
contamine1 = pd.read_csv(path1 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered1 = pd.read_csv(path1 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)
contamine2 = pd.read_csv(path2 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered2 = pd.read_csv(path2 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)

N = 50000 # S + I = N
S = N - 1
I = 1
R = 0
beta = 0.5
gamma = 0.1
iterations = 100

S_comp = [] # infected compartment
I_comp = [] # susceptible compartment
R_comp = [] # recovered compartment

infection(S, I, R, N, iterations, I_comp, S_comp, R_comp, beta, gamma)
figure = plt.figure(figsize=(15, 15))

# Infected
plt.subplot(3,1,1)
I_SIR, = plt.plot(I_comp, label='I_SIR(t)')
I_50000, = plt.plot(contamine1, label='I_50000(t)')
I_MIX, = plt.plot(contamine2, label='I_MIX(t)')

plt.title("SIR_référence, densité 1/16, 50000 individus, Compartiment Infected")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[I_SIR, I_50000, I_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Recovered
plt.subplot(3,1,2)
R_SIR, = plt.plot(R_comp, label='R_SIR(t)')
R_50000, = plt.plot(recovered1, label='R_50000(t)')
R_MIX, = plt.plot(recovered2, label='R_MIX(t)')

plt.title("SIR_référence, densité 1/16, 50000 individus, Compartiment Recovered")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[R_SIR, R_50000, R_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Susceptible
plt.subplot(3,1,3)
S_SIR, = plt.plot(S_comp, label='S_SIR(t)')
S_50000, = plt.plot((N-contamine1)['Nombre de contaminés'].subtract(recovered1['Recovered']), label='S_50000(t)')
S_MIX, = plt.plot((N-contamine2)['Nombre de contaminés'].subtract(recovered2['Recovered']), label='S_MIX(t)')

plt.title("SIR_référence, densité 1/16, 50000 individus, Compartiment Susceptible")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[S_SIR, S_50000, S_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

plt.show()

### MSE

In [ ]:
susceptible1 = (N-contamine1)['Nombre de contaminés'].subtract(recovered1['Recovered'])
mse1 = SIR_mse_calc(N,contamine1,susceptible1,recovered1)
print("MSE minimal (SIR_5000,SIR_SIR) : ", mse1[0], " pour un paramètre beta = ", mse1[1], " et pour un paramètre gamma = ", mse1[2])

susceptible2 = (N-contamine2)['Nombre de contaminés'].subtract(recovered2['Recovered'])
mse2 = SIR_mse_calc(N,contamine2,susceptible2,recovered2)
print("MSE minimal (SIR_MIX,SIR_SIR) : ", mse2[0], " pour un paramètre beta = ", mse2[1], " et pour un paramètre gamma = ", mse2[2])

# Adapt length
if(len(contamine1) < len(contamine2)):
    contamine2 = contamine2[:len(contamine1)]
    recovered2 = recovered2[:len(recovered1)]
    susceptible2 = susceptible2[:len(susceptible1)]
else:
    contamine1 = contamine1[:len(contamine2)]
    recovered1 = recovered1[:len(recovered2)]
    susceptible1 = susceptible1[:len(susceptible2)]

print("MSE (I_5000,I_MIX) : ", np.square(np.subtract(contamine1['Nombre de contaminés'].values.tolist(),contamine2['Nombre de contaminés'].values.tolist())).mean() + np.square(np.subtract(recovered1['Recovered'].values.tolist(),recovered2['Recovered'].values.tolist())).mean() + np.square(np.subtract(susceptible1.values.tolist(),susceptible2.values.tolist())).mean())

### 100000 individus

In [ ]:
path1 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/5000mouvements/densite1sur16/population100000"
path2 = str(Path("./").parent.absolute().parents[0]) + "/Simulations/SIR/Reference/perfect_mix/densite1sur16/population100000"

In [ ]:
temps = pd.read_csv(path1 + "/data_csv/times.csv")
temps_data = pd.DataFrame(temps)
print(temps_data)

In [ ]:
contamine1 = pd.read_csv(path1 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered1 = pd.read_csv(path1 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)
contamine2 = pd.read_csv(path2 + "/data_csv/m_nombre_contamine.csv", dtype=int)
recovered2 = pd.read_csv(path2 + "/data_csv/m_SIR_recovered_file.csv", dtype=int)

N = 5000 # S + I = N
S = N - 1
I = 1
R = 0
beta = 0.5
gamma = 0.1
iterations = 100

S_comp = [] # infected compartment
I_comp = [] # susceptible compartment
R_comp = [] # recovered compartment

infection(S, I, R, N, iterations, I_comp, S_comp, R_comp, beta, gamma)
figure = plt.figure(figsize=(15, 15))

# Infected
plt.subplot(3,1,1)
I_SIR, = plt.plot(I_comp, label='I_SIR(t)')
I_100000, = plt.plot(contamine1, label='I_100000(t)')
I_MIX, = plt.plot(contamine2, label='I_MIX(t)')

plt.title("SIR_référence, densité 1/16, 100000 individus, Compartiment Infected")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[I_SIR, I_100000, I_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Recovered
plt.subplot(3,1,2)
R_SIR, = plt.plot(R_comp, label='R_SIR(t)')
R_100000, = plt.plot(recovered1, label='R_100000(t)')
R_MIX, = plt.plot(recovered2, label='R_MIX(t)')

plt.title("SIR_référence, densité 1/16, 100000 individus, Compartiment Recovered")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[R_SIR, R_100000, R_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

# Susceptible
plt.subplot(3,1,3)
S_SIR, = plt.plot(S_comp, label='S_SIR(t)')
S_100000, = plt.plot((N-contamine1)['Nombre de contaminés'].subtract(recovered1['Recovered']), label='S_100000(t)')
S_MIX, = plt.plot((N-contamine2)['Nombre de contaminés'].subtract(recovered2['Recovered']), label='S_MIX(t)')

plt.title("SIR_référence, densité 1/16, 100000 individus, Compartiment Susceptible")
plt.xlabel("Itérations")
plt.ylabel("Individus")
plt.legend(handles=[S_SIR, S_100000, S_MIX])

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0)) # use scientific notation
plt.xlim([0, iterations])

plt.show()

# MSE I
#print("MSE I : ", np.square(np.subtract(contamine['Nombre de contaminés'].values.tolist(),I_comp)).mean())

# MSE S
#print("MSE S : ", np.square(np.subtract(nombre_individus_data_S.values.tolist(),S_comp)).mean())

# MSE R
#print("MSE R : ", np.square(np.subtract(recovered['Recovered'].values.tolist(),R_comp)).mean())

### MSE

In [ ]:
susceptible1 = (N-contamine1)['Nombre de contaminés'].subtract(recovered1['Recovered'])
mse1 = SIR_mse_calc(N,contamine1,susceptible1,recovered1)
print("MSE minimal (SIR_5000,SIR_SIR) : ", mse1[0], " pour un paramètre beta = ", mse1[1], " et pour un paramètre gamma = ", mse1[2])

susceptible2 = (N-contamine2)['Nombre de contaminés'].subtract(recovered2['Recovered'])
mse2 = SIR_mse_calc(N,contamine2,susceptible2,recovered2)
print("MSE minimal (SIR_MIX,SIR_SIR) : ", mse2[0], " pour un paramètre beta = ", mse2[1], " et pour un paramètre gamma = ", mse2[2])

# Adapt length
if(len(contamine1) < len(contamine2)):
    contamine2 = contamine2[:len(contamine1)]
    recovered2 = recovered2[:len(recovered1)]
    susceptible2 = susceptible2[:len(susceptible1)]
else:
    contamine1 = contamine1[:len(contamine2)]
    recovered1 = recovered1[:len(recovered2)]
    susceptible1 = susceptible1[:len(susceptible2)]

print("MSE (I_5000,I_MIX) : ", np.square(np.subtract(contamine1['Nombre de contaminés'].values.tolist(),contamine2['Nombre de contaminés'].values.tolist())).mean() + np.square(np.subtract(recovered1['Recovered'].values.tolist(),recovered2['Recovered'].values.tolist())).mean() + np.square(np.subtract(susceptible1.values.tolist(),susceptible2.values.tolist())).mean())